## Load packages

In [9]:
import re
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score,recall_score

## 1 - Preparation

### 1.1 - Load local data

In [2]:
# raw_data = pd.read_excel("NLP_Sample_dataset.xlsx")
# raw_data = raw_data[['Code', 'jd']]
# raw_data

### 1.2 - Load data from PostgreSQL database

In [2]:
# Test connecttion to the PostgreSQL sever
def test_sql_connection():
    try:
        conn = psycopg2.connect(dbname="postgres", user="readonly",
                                host="47.103.73.205", password="123456", port="25432")
        print("connection successful")
    except:
        print("I am unable to connect to the database")
    conn.close()

test_sql_connection()

connection successful


In [3]:
# Load data from PostgreSQL sever
def load_data(sql_command):
    conn = psycopg2.connect(dbname="postgres", user="readonly",
                                host="47.103.73.205", password="123456", port="25432")
    data = pd.read_sql(sql_command, conn) # Read data as dataframe
    conn.close()
    
    return data

**Test function:** `load_data(sql_command)` 

In [4]:
test_sql = "SELECT * FROM meta_training_data_jd LIMIT 5;"
load_data(test_sql)

,Type,Code,fun1_code,fun2_code,jobname_ch,jobname_en,id,coname,poname,jd,source
0,DIS,APM010,APM,010,信息技术项目管理,Information Technology Project Management,45469,Savi Technologies,Sr. IT Project Manager,Savi Technologies Inc. is seeking Sr. IT Proje...,indeed
1,DIS,APM010,APM,010,信息技术项目管理,Information Technology Project Management,45472,Albemarle Corporation,Sr. IT Project Manager,Are you inspired by “what’s next”? So are we.W...,indeed
2,DIS,APM010,APM,010,信息技术项目管理,Information Technology Project Management,45475,The American Kennel Club Inc,IT Project Manager,I. Position DescriptionThis position is respon...,indeed
3,DIS,APM010,APM,010,信息技术项目管理,Information Technology Project Management,45482,Avani Technology Solutions Inc,IT Project Manager,Job DetailsJob CodeJPSC-6965Posted Date03/08/1...,indeed
4,DIS,APM010,APM,010,信息技术项目管理,Information Technology Project Management,45483,Plymouth Rock Assurance,IT Project Manager,Position Summary:This person will perform Proj...,indeed


### 1.3 - Define functions to generate the feature matrix with word count
`CountVectorizer` library: [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [5]:
# get stop words
stopwords = []
with open("stop_words_en.txt", 'r') as file:
    for word in file.readlines():
        stopwords.append(word.strip('\n'))

        
# clean the feature columns, keep only pure english words        
def clean_feature(df):
    feature_list = df.columns.tolist()
    
    word_feature = []
    for i in feature_list:
        match = re.findall(r'^[a-z]+$', i)
        if match:
            word_feature.append(i)
        else: 
            pass
    
    return df[word_feature]


# Generate feature matrix
def generate_feature_matrix(text, label):
    """Input: ('text', 'label': list or array-like), generate a feature matrix with word count and label)"""
    cv = CountVectorizer(analyzer='word', stop_words=stopwords)
    cv_fit = cv.fit_transform(text)

    feature_matrix = pd.DataFrame(cv_fit.toarray(), columns=cv.get_feature_names())
    feature_matrix = clean_feature(feature_matrix)  # Clean feature name
    feature_matrix.insert(0, "data_label", label)  # Add label

    return feature_matrix

**Test function:** `generate_feature_matrix`

In [6]:
# Test text
test_text = ["Data science is an inter-disciplinary field to extract knowledge and insights from data.",
             "Data science is related to data mining and big data 666 555teststring.",
             "Data science unifies statistics, data analysis, and machine learning."]
test_label = ["Joseph", "is" ,"awesome"]

# Test
test_matrix = generate_feature_matrix(test_text, test_label)
test_matrix

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,data_label,analysis,data,disciplinary,extract,field,insights,inter,knowledge,learning,machine,mining,science,statistics,unifies
0,Joseph,0,2,1,1,1,1,1,1,0,0,0,1,0,0
1,is,0,3,0,0,0,0,0,0,0,0,1,1,0,0
2,awesome,1,2,0,0,0,0,0,0,1,1,0,1,1,1


In [7]:
def model_evaluation(model, X_test, y_test):
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    balanced = balanced_accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average='micro')
    specificity = recall_score(y_test, y_pred, pos_label=0, average='micro')

    print('Accaccuracy: {}\nRecall: {}\nSpecificity: {}\nBalanced: {}'.format(
        acc, recall, specificity, balanced))

## 2 - Model training

### 2.1 - Prepare train-test data 

In [8]:
%%time

sql_1 = 'SELECT "Code", "jd" FROM meta_training_data_jd LIMIT 2000;'
all_data = load_data(sql_1)

all_data.info()

""" **Time warning**
1,000 reocods: 24.9 s 
10,000 records: 4 min 50s
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Code    2000 non-null   object
 1   jd      2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB
CPU times: user 64.9 ms, sys: 86.1 ms, total: 151 ms
Wall time: 53.4 s


' **Time warning**\n1,000 reocods: 24.9 s \n10,000 records: 4 min 50s\n'

In [13]:
data = generate_feature_matrix(all_data.jd, all_data.Code)
data

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,data_label,aa,aaa,aabb,aac,aace,aadditional,aae,aais,aanderson,...,zp,zr,zug,zumajob,zuniga,zunigahenry,zuora,zweig,zyberk,zycus
0,APM010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,APM010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,APM010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,APM010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,APM010,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,AFB000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,AFB000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,AFB000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,AFB000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
data_sort = data.iloc[:,data.columns != "data_label"]

In [44]:
print(data_sort.sum().sort_values())

llcis                      1
obstaclesmay               1
obstaclesestablishes       1
obstacle                   1
obsessive                  1
                        ... 
quality                 3603
team                    3771
management              5146
experience              6420
project                 7820
Length: 32899, dtype: int64


[train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [18]:
X = data.iloc[:,data.columns != "data_label"] 
y = data.iloc[:,data.columns == "data_label"]

# Split train/test data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7) #, stratify=y)

In [19]:
# X = data.iloc[:,data.columns != "data_label"] 
# y = data.iloc[:,data.columns == "data_label"]

# # Split train/test data
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7, stratify=y)

In [12]:
# # Reindex
# for i in [X_train, X_test, Y_train, Y_test]:
#     i.index = range(i.shape[0])

### 2.2 - Random Forest test
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [20]:
%%time

# Establish random forest model
rfc = RandomForestClassifier(n_estimators=1000, random_state=0)

# Model training
rfc.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


CPU times: user 1min 8s, sys: 448 ms, total: 1min 8s
Wall time: 1min 8s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [22]:
model_evaluation(rfc, X_test, y_test)

Accaccuracy: 0.47833333333333333
Recall: 0.47833333333333333
Specificity: 0.47833333333333333
Balanced: 0.20764018796623418


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1259: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


### 2.3 - XGB test
https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn

In [15]:
%%time

params = {'silent': True, 'n_estimators': 100, 'objective': 'multi:softmax'}

xgbc = XGBClassifier(**params)
xgbc.fit(X_train,y_train,eval_metric='auc')

# Wall time: 21min 37s

/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


CPU times: user 3h 2min 57s, sys: 2min 15s, total: 3h 5min 13s
Wall time: 25min 1s


XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, silent=True, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [18]:
model_evaluation(xgbc, X_test, y_test)

Accaccuracy: 0.72
Recall: 0.72
Specificity: 0.72
Balanced: 0.5414963052854933


## Test TF-IDF

In [27]:
all_data

,Code,jd
0,APM010,Savi Technologies Inc. is seeking Sr. IT Proje...
1,APM010,Are you inspired by “what’s next”? So are we.W...
2,APM010,I. Position DescriptionThis position is respon...
3,APM010,Job DetailsJob CodeJPSC-6965Posted Date03/08/1...
4,APM010,Position Summary:This person will perform Proj...
...,...,...
1995,AFB000,Provides financial analytical support by colle...
1996,AFB000,"KION North America is part of the KION Group, ..."
1997,AFB000,"Who We Are:Pulse Films is an award-winning, in..."
1998,AFB000,"Overview:ERP International, LLC (www.erpintern..."


In [28]:
tv = TfidfVectorizer(analyzer='word', stop_words=stopwords)
tv_fit = tv.fit_transform(all_data.jd)
tv_fit

# tv_fit.toarray()
# type(tv_fit)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'al', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


<2000x35426 sparse matrix of type '<class 'numpy.float64'>'
	with 404250 stored elements in Compressed Sparse Row format>

In [34]:
# tv.get_feature_names()

In [30]:
tv_X = tv_fit
tv_y = all_data.Code

# Split train/test data
X_train, X_test, y_train, y_test = train_test_split(tv_X, tv_y, random_state=42, train_size=0.7) #, stratify=y)

In [31]:
%%time

# Establish random forest model
tv_rfc = RandomForestClassifier(n_estimators=1000, random_state=0)

# Model training
tv_rfc.fit(X_train, y_train)

CPU times: user 26.2 s, sys: 293 ms, total: 26.5 s
Wall time: 26.5 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [32]:
model_evaluation(tv_rfc, X_test, y_test)

Accaccuracy: 0.4866666666666667
Recall: 0.4866666666666667
Specificity: 0.4866666666666667
Balanced: 0.19293152003352773


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1259: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
